<a href="https://colab.research.google.com/github/lookinsight/ml/blob/main/20221115_ML_XGBoost_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XGBoost - 커플 성사 여부 예측

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# https://www.kaggle.com/datasets/annavictoria/speed-dating-experiment
file_url = 'https://raw.githubusercontent.com/bigdata-young/bigdata_16th/main/data/dating.csv'
df = pd.read_csv(file_url)

In [ ]:
df.head()

In [ ]:
pd.options.display.max_columns = 40 # 총 40개 컬럼까지 출력되도록 설정

In [ ]:
df.head()

In [ ]:
df.info() # 결측치와 변수 타입

## 변수 목록
* has_null
    - 변수 중 Null값이 있는지 여부. 단, 이 데이터는 기존 데이터에서 일부 변수들이 생략된 축소판이기 때문에, 여기서 보이는 Null값 여부와 다소 차이가 있을 수 있음.
    - 전반적으로 무응답 항목이 있는지에 대한 정보이므로 그대로 사용
* age / age_o : age는 본인 나이이며 age_o는 상대방 나이.
* race / race_o : 마찬가지로 본인과 상대의 인종 정보.
* importance_same_race / importance_same_religion
    * 인종과 종교를 중요시 여기는지에 대한 응답
* attractive(매력적인), sincere(성실한), intelligence(지적), funny(재미난), ambitious(야심찬), shared_ interests(공통관심사) : 이 항목들은 4가지 관점에서 평가되어 총 변수가 24(6 × 4)개
    * pref_o_xxx( 예 : pref_o_attractive) : 상대방이 xxx 항목을 얼마나 중요하게 생각하는가에 대한 응답
    * xxx_o(예: attractive_o) : 상대방이 본인에 대한 xxx 항목을 평가한 항목
    * xxx_important(예 : attractive_important) : xxx 항목에 대해 본인이 얼마나 중요하게 생각하는가에 대한 응답
    * xxx_partner(예 : attractive_partner) : 본인이 상대방에 대한 xxx 항목을 평가한 항목
* interests_correlate : 관심사(취미 등) 연관도
* expected_happy_with_sd_people : 스피드 데이팅을 통해 만난 사람과 함께할 때 얼마나 좋을
지에 대한 기대치
* expected_num_interested_in_me : 얼마나 많은 사람이 나에게 관심을 보일지에 대한 기대치
* like : 파트너가 마음에 들었는지 여부
* guess_prob_liked : 파트너가 나를 마음에 들어했을지에 대한 예상
* met: 파트너를 스피드 데이팅 이벤트 이전에 만난 적이 있는지 여부

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
df.describe()

# 전처리

## 결측치

In [ ]:
# df.isna().mean()
df.isna().mean().sort_values(ascending = False) 

In [ ]:
df = df.dropna(
    subset=['pref_o_attractive', 'pref_o_sincere', 'pref_o_intelligence',
            'pref_o_funny', 'pref_o_ambitious', 'pref_o_shared_interests',
            'attractive_important', 'sincere_important', 'intellicence_important',
            'funny_important', 'ambtition_important', 'shared_interests_important'])
# 일부 변수에서 결측치 제거

In [ ]:
df.fillna(-99, inplace = True) 

In [ ]:
df.isna().mean().sort_values(ascending=False)

### 피처 엔지니어링

- 피처(Feature) == 독립변수들...  / 엔지니어링 -> 가공해서 더 유의미하게 쓰겠다

- 나이? 중요도? -> 계산 -> 합쳐주거나, 새로운 변수화

### 나이

In [ ]:
# apply(axis = 1) 
# df.age = 본인 나이, df.age_o = 상대방 나이 
def age_gap(x): # 행 전체
    if x['age'] == -99:  # 내 나이가 결측치면
        return -99       # 나이 차이도 결측치 
    if x['age_o'] == -99:   # 상대방 나이도 결측치면 
        return -99
    if x['gender'] == 'female':
        return x['age_o'] - x['age']  # 상대방 나이가 얼마나 더 많은지 (여성) 
    if x['gender'] == 'male':
        return x['age'] - x['age_o']   # 내가 상대방보다 나이가 얼마나 많은지 (남성) 


In [ ]:
# 나이차이 + 성별간의 차이 
df['age_gap'] = df.apply(age_gap, axis=1)
df.age_gap.head()

In [ ]:
# df['age_gap'].plot()

In [ ]:
# 나이 차이만 (절대값) 
df['age_gap_abs'] = abs(df.age_gap)
df.age_gap_abs.unique()

In [ ]:
df.race.unique()
df.race_o.unique()

In [ ]:
# df.race, df.race_o 
def same_race(x):
    if x['race'] == -99: return -99 
    if x.race_o == -99: return -99 
    if x.race == x.race_o: return 1
    return -1 

In [ ]:
df['same_race'] = df.apply(same_race, axis = 1) 
df.same_race.unique() 

In [ ]:
df.importance_same_race.value_counts() 

In [ ]:
def same_point(x):  # apply(axis = 1) 
    if x['same_race'] == -99:             # 결측치면
        return -99                        # 결측치로 두고
    # 1, -1 
    return x['same_race'] * x['importance_same_race'] 

In [ ]:
df['same_race_point'] = df.apply(same_point, axis = 1) 
# df.same_race_point.head()
df.same_race_point.value_counts() 

In [ ]:
df[['race','race_o', 'same_race','importance_same_race','same_race_point']]

In [ ]:
# 중요도 * 점수 => 파생변수(함수) 
# importance(중요하게 여기는 대상), score(그 대상에 얼마나 점수를 주는지) -> 컬럼 이름 
# data = 행 (row) 
def rating(data, importance, score):         # 점수를 부여하는 함수
    if data[importance] == -99: return -99   # 결측치
    if data[score] == -99: return -99      # 결측치
    return data[importance] * data[score]    # 중요한 것 * 그것에 대한 점수

In [ ]:
df.columns

In [ ]:
df.columns[8:14]   # 상대방의 나에 대한 선호도 (8 ~ 13) 

In [ ]:
df.columns[14:20]    # 본인에 대한 상대방의 평가 (14  ~19)  - 상대방이 이 분류에 대해 나를 어떻게 생각하는지 

In [ ]:
df.columns[20:26]   # 나(본인)의 중요도 - 그 특성에 대한 

In [ ]:
df.columns[26:32]   # 상대방에 대한 본인의 평가 

In [ ]:
print(f'상대방의 선호도 : {df.columns[8:14]}')
print(f'본인에 대한 상대방의 평가 : {df.columns[14:20]}')
print(f'본인의 선호도 : {df.columns[20:26]}')
print(f'상대방에 대한 본인의 평가 : {df.columns[25:32]}')

In [ ]:
partner_imp = df.columns[8:14]
partner_rate_me = df.columns[14:20]
my_imp = df.columns[20:26]
my_rate_partner = df.columns[25:32]

In [ ]:
new_label_partner = ['attrative_p', 'sinsere_partner_p', 'intelligence_p',
                     'funny_p', 'ambition_p','shared_interests_p']
new_label_me = ['attrative_m', 'sinsere_partner_m', 'intelligence_m',
                     'funny_m', 'ambition_m','shared_interests_m']                

In [ ]:
# 평가점수 * 중요도 => 새로운 라벨 
for i, j, k in zip(new_label_partner, partner_imp, partner_rate_me,):
    print(f'{i} & {j} & {k}')

In [ ]:
# 파트너가 나에게 느끼는 점수 / 상대방의 선호도 / 나에 대한 파트너의 평가 
for i, j, k in zip(new_label_partner, partner_imp, partner_rate_me):
    # i => new_abel_partner (새로운 컬럼 이름) -> df[i]
    # j => 상대방의 특정 영역에 대한 선호도(importance) 
    # k => 나에 대한 파트너 평가 (score) 
    df[i] = df.apply(lambda x: rating(x, j, k), axis = 1)

In [ ]:
df.columns

In [ ]:
# 파트너가 나에게 느끼는 점수 / 상대방의 선호도 / 나에 대한 파트너의 평가 
for i, j, k in zip(new_label_me, my_imp, my_rate_partner):
    # i => new_label_me (새로운 컬럼 이름) -> df[i]
    # j => 나의 상대방 특정 영역에 대한 선호도(importance) 
    # k => 상대방에 대한 나의 평가 (score) 
    df[i] = df.apply(lambda x: rating(x, j, k), axis = 1)

In [ ]:
df.columns

### 범주형 변수 변환

In [ ]:
df.info()

In [ ]:
df.describe(include=['O']) 

In [ ]:
df = pd.get_dummies(df, columns = ['gender','race','race_o'], drop_first = True)

In [ ]:
df.info()

## 모델링 및 평가

In [ ]:
#@title 훈련셋 / 시험셋
from sklearn.model_selection import train_test_split 

X = df.drop('match', axis = 1) 
y = df.match 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 100) 

### 모델링

In [ ]:
import xgboost as xgb

In [ ]:
# 모델 객체
model = xgb.XGBClassifier(n_estimators = 500, max_depth = 5, random_state = 100) 

In [ ]:
model.fit(X_train, y_train) 

In [ ]:
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# 매칭 여부
# df.match.mean()
1 - df.match.mean()

In [ ]:
accuracy_score(y_test, pred) 

In [ ]:
# 오른쪽 위 : 1종 오류 - 실제 틀린데 (0) 맞다고 예측 (1) - 68
# 왼쪽 아래 : 2종 오류 - 실제 맞는데 (1) 틀리다고 예측 (0) - 147
print(confusion_matrix(y_test, pred))
cf_matrix = confusion_matrix(y_test,pred)
cf_matrix
group_names = ['TN','FP','FN','TP']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='coolwarm')
plt.ylabel('True')
plt.xlabel('Predicted')
plt.show()
# TP(양성을 양성으로 판단)
# FN(양성을 음성으로 판단) -> 2종 오류 (맞는데 틀리다고)
# FP(음성을 양성으로 판단) -> 1종 오류 (틀린데 맞다고)
# TN(음성을 음성으로 판단) 

In [ ]:
#@title classification_report()
# - 종속변수의 값인 0과 1 각각에 대하여 나타남
# - 예측하려는 경우(1)에 대한 값을 주로 해석하면 됨
print(classification_report(y_test, pred))
# precision : 정밀도
# recall : 재현율
# f1-score : F1-점수
# support : 인덱스

#### 정밀도(precision)
* 1로 예측한 경우 중, 얼마만큼이 실제로 1인지를 나타냄<br>
$TP\over(TP+FP)$
=
$\frac{양성을 양성으로 판단}{양성을 양성으로 판단 + 1종 오류}$
=
$\frac{양성을 양성으로 판단}{양성으로 판단한 수}$
> FP가 커질수록 분모가 커지기 때문에 정밀도는 낮아짐 (1종 오류와 관련)

#### 재현율(recall)
* 실제로 1 중에, 얼마만큼을 1로 예측했는지 나타냄<br>
$TP\over(TP+FN)$
=
$\frac{양성을 양성으로 판단}{양성을 양성으로 판단 + 2종 오류}$
=
$\frac{양성을 양성으로 판단}{실제로 양성인 수}$
> FN가 커질수록 분모가 커지기 때문에 재현율은 낮아짐 (2종 오류와 관련)

#### F-1점수(f-score)
* 정밀도와 재현율의 조화평균<br>
* 조화평균 : 주어진 수들의 역수의 산술평균의 역수
    * 예) $H = \frac{2a_1a_2}{a_1 + a_2}$

$2 \times \frac{precision \times recall}{precesion + recall}$
=
$2 \times \frac{정밀도 \times 재현율}{정밀도 + 재현율}$

> 1종 오류가 중요하면 정밀도, 2종 오류가 중요하면 재현율<br>
> 딱히 중요한 것 없으면 F1-점수

## 하이퍼파라미터 튜닝

### 경사하강법
* 머신 러닝이 학습시킬 때 최소의 오차를 찾는 방법
* 오차 함수에 대한 경사도(미분계수)를 기준으로 매개변수를 반복적으로 이동해가며 최소 오차를 찾음
* 매개변수? : 선형 회귀에서의 계수(변수에 대한 기울기 값)에 해당

**경사하강법과 보폭**
* 경사부스팅의 핵심개념 중 하나로, 모델이 어떻게 최소 오차가 되는 매개변수들을 학습하는지에 대한 방법
* 오차식에 대한 미분계수를 통해 매개 변수의 이동 방향과 보폭을 결정합니다
* 보폭은 매개변수를 얼만큼씩 이동할지를 의미

**미분계수**
* 평균변화율에서 x의 증가량을 0으로 가깝게 할 때의 평균변화율
* 그래프 상에서 접선의 기울기, 계수

### Gridsearch(그리드서치) 

* learning_rate
    * 경사하강법에서 ‘매개변수’를 얼만큼씩 이동해가면서 최소 오차를 찾을지, 그 보폭의 크기를 결정하는 하이퍼파라미터.
    * 기본적으로 보폭은 미분계수에 의해 결정되지만, learning_rate를 크게 하면 더 큰 보폭을, 작게 하면 그만큼 작은 보폭으로 움직임.
    * learning rate를 우리말로 학습률이라고 함
    * 학습률과 보폭 : 학습률은 입력, 보폭은 그 결과. 큰 학습률을 사용하면 결과적으로 보폭도 커짐
    * 너무 작은 학습률 -> 상당한 시간이 들고, 오버피팅 문제 일어남
    * 너무 큰 학습률 -> 학습이 제대로 안 됨
    * 적절한 크기의 학습률을 사용해야만 큰 시간을 들이지 않고 최소 오차 지점을 찾을 수 있음
• max_depth : 각 트리의 깊이를 제한
• subsample : 모델을 학습시킬 때 일부 데이터만 사용하여 각 트리를 만듦. 0.5를 쓰면 데
이터의 절반씩만 랜덤 추출하여 트리를 만듦. 이 또한 오버피팅을 방지하는 데 도움이 됨
• n_estimators : 전체 나무의 개수.

In [ ]:
parameter = {
    'learning_rate': [0.01, 0.1, 0.3], # 경사하강법 : '매개변수' -> 최소오차 -> 보폭 크기
    'max_depth': [5, 7, 10], # 트리의 깊이 (오버피팅)
    'subsample': [0.5, 0.7, 1], # 추출할 데이터 비율
    'n_estimators': [300, 500, 1000] # 트리 개수
}

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model = xgb.XGBClassifier()

In [ ]:
gs_model = GridSearchCV(model, parameter, n_jobs=-1, scoring='f1', cv = 5)

In [ ]:
gs_model.fit(X_train, y_train)

In [ ]:
!pip install mlxtend --quiet

import joblib
joblib.dump(gs_model, 'gs_model.pkl')

In [ ]:
gs_model.best_params_

In [ ]:
pred = gs_model.predict(X_test)

In [ ]:
print(f"accuracy_score : {accuracy_score(y_test, pred)}")
print(classification_report(y_test, pred))

## 변수의 영향력 -> 중요 변수 확인

In [ ]:
model = xgb.XGBClassifier(learning_rate=0.1, max_depth=5, 
                          n_estimators= 500, subsample=1, random_state=100)
model.fit(X_train, y_train)

In [ ]:
# 중요한 변수
model.feature_importances_

In [ ]:
feature_imp = pd.DataFrame({'features': X_train.columns, 'values': model.feature_importances_})

In [ ]:
pd.options.display.float_format = '{:.6f}'.format
feature_imp.head()

In [ ]:
feature_imp.sort_values(by='values', ascending=False)

In [ ]:
plt.figure(figsize=(100, 200))
feature_imp.sort_values(by='values', ascending=True).plot(kind='bar')

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x='values', y='features',
            data=feature_imp.sort_values(by='values', ascending=False).head(10))